In [2]:
from typing import Optional, List
from bs4 import BeautifulSoup, Tag
import requests

def convert_updown_number(text) -> float:
    if text.find('보합') == 0:
        return 0.0

    tokens = text.split()
    if tokens[0] == '하락':
        return float(tokens[1]) * -1
    else :
        return float(tokens[1])

tx = '''하락
				425'''
print(convert_updown_number(tx))

def convert_number(text) -> float:
    negative = 1
    if text[0] == '+':
        text = text[1:]
        negative = 1
    elif text[0] == '-':
        text = text[1:]
        negative = -1

    if text.endswith('%'):
        text = text[:-1]

    text = text.replace(',', '')
    return float(text) * negative

print(convert_number('12,123,123'))

class InvestorVolume:
    def __init__(self, date, personal, foreigner, company, finance, insurance, toosin, bank, etc_fin, gov_fund, etc_comp):
        self.date = date
        self.personal = personal
        self.foreigner = foreigner
        self.company = company
        self.finance = finance
        self.insurance = insurance
        self.toosin = toosin
        self.bank = bank
        self.etc_fin = etc_fin
        self.gov_fund = gov_fund
        self.etc_comp = etc_comp

def crawl_investors(url) -> List[InvestorVolume]:
#url = 'https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=&page=1'
    req = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    dom = BeautifulSoup(req.text, "html.parser")
    sel = dom.select('table.type_1 > tr > td.date2')

    sel_c: List[InvestorVolume] = []

    for trd in sel:
        tr = trd.parent
        sel_c.append(InvestorVolume(
            tr.select_one('td:nth-child(1)').text,
            convert_number(tr.select_one('td:nth-child(2)').text),
            convert_number(tr.select_one('td:nth-child(3)').text),
            convert_number(tr.select_one('td:nth-child(4)').text),
            convert_number(tr.select_one('td:nth-child(5)').text),
            convert_number(tr.select_one('td:nth-child(6)').text),
            convert_number(tr.select_one('td:nth-child(7)').text),
            convert_number(tr.select_one('td:nth-child(8)').text),
            convert_number(tr.select_one('td:nth-child(9)').text),
            convert_number(tr.select_one('td:nth-child(10)').text),
            convert_number(tr.select_one('td:nth-child(11)').text)
        ))
    return sel_c

url = 'https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=&page=1'

for a in crawl_investors(url):
    print(a.__dict__)

-425.0
12123123.0
{'date': '24.07.26', 'personal': -3730.0, 'foreigner': -475.0, 'company': 3983.0, 'finance': 2857.0, 'insurance': 91.0, 'toosin': 1167.0, 'bank': -204.0, 'etc_fin': 21.0, 'gov_fund': 51.0, 'etc_comp': 222.0}
{'date': '24.07.25', 'personal': 8267.0, 'foreigner': -6785.0, 'company': -1529.0, 'finance': -3.0, 'insurance': -7.0, 'toosin': -1247.0, 'bank': -139.0, 'etc_fin': 82.0, 'gov_fund': -216.0, 'etc_comp': 48.0}
{'date': '24.07.24', 'personal': 2345.0, 'foreigner': -2654.0, 'company': 410.0, 'finance': 62.0, 'insurance': 165.0, 'toosin': 382.0, 'bank': -51.0, 'etc_fin': -21.0, 'gov_fund': -127.0, 'etc_comp': -101.0}
{'date': '24.07.23', 'personal': -1990.0, 'foreigner': 2502.0, 'company': -246.0, 'finance': -1064.0, 'insurance': 86.0, 'toosin': 769.0, 'bank': -137.0, 'etc_fin': -35.0, 'gov_fund': 137.0, 'etc_comp': -266.0}
{'date': '24.07.22', 'personal': 1414.0, 'foreigner': -845.0, 'company': -238.0, 'finance': 1520.0, 'insurance': -73.0, 'toosin': -703.0, 'bank': 

In [14]:
import pymysql

def crawl2db(from_date='20240726', market_code = '01', page = 1):
    target_url = 'https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=%s&sosok=%s&page=%s' % (from_date, market_code, page)
    print(target_url)
    data = crawl_investors(target_url)

    conn = pymysql.connect(host='horusa', user='root', password='root', db='zio', charset='utf8mb4')
    cur = conn.cursor()
    
    sql = "INSERT INTO CRAWL_INVESTOR_DAILY_VOLUME(TYPE_CODE, DATEON, PERSONAL, FOREIGNER, COMPANY, FINANCE, INSURANCE, TOOSIN, BANK, ETC_FIN, GOV_FUND, ETC_FUND) VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s); "

    # self.date = date
    # self.personal = personal
    # self.foreigner = foreigner
    # self.company = company
    # self.finance = finance
    # self.insurance = insurance
    # self.toosin = toosin
    # self.bank = bank
    # self.etc_fin = etc_fin
    # self.gov_fund = gov_fund
    # self.etc_comp = etc_comp
    
    for row in data:
        try:
            cur.execute(sql, (market_code, row.date, row.personal, row.foreigner, row.company, row.finance, row.insurance, row.toosin, row.bank, row.etc_fin, row.gov_fund, row.etc_comp))
        except:
            print('error ---> ' + row.__dict__)

    conn.commit()
    conn.close()
    
crawl2db()    

https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=1


In [15]:
# 2 to 484
import time
for i in range(2, 484):
    print('-------> turn #{}'.format(i))
    crawl2db(page = i)
    time.sleep(1)



-------> turn #2
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=2
-------> turn #3
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=3
-------> turn #4
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=4
-------> turn #5
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=5
-------> turn #6
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=6
-------> turn #7
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=7
-------> turn #8
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=8
-------> turn #9
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=9
-------> turn #10
https://finance.naver.com/sise/investorDealTrendDay.naver?bizdate=20240726&sosok=01&page=10
-------> turn #11
https://